# Generate Random Combis
In this notebook we randomly generate new combis using those calls found in the recordings. Thereby, we create a dataframe containing new combis wherein call probability of occurence is the same as in the original dataset, however the  probability of transition between calls is random. We repeat this code for 10, 000 iterations to produce a large sample size of randomly generated combi arrays. We then calculate call transition probability within each of these randomly generated combi arrays. 

In [1]:
from tqdm.auto import tqdm

In [2]:
import avgn

In [3]:
import pandas as pd
import numpy as np
from avgn.utils.paths import DATA_DIR, ensure_dir, FIGURE_DIR

In [4]:
from scipy.stats import kruskal

In [5]:
DATASET_ID = "git_repos_call"

In [6]:
DT_ID = '2022-03-12_17-46-00'

In [7]:
call_df = pd.read_pickle(DATA_DIR / DATASET_ID / DT_ID /  'call_umap_grouped.pickle')
call_df[:3]

,start_time,end_time,labels,indv,indvi,filename,group,location,sex,wav_loc,...,comb_labels,umap,spectrogram,call_lab_simp,combi_lab_simp,call_unique_num,call_pos_combi,combi_label,combi_unique_num,simp
ID,,,,,,,,,,,,,,,,,,,,,
0,0.753604,0.921160,DSSHDS,MGGY,0,BWY MGGY Call Combo 1 290719 PM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,DSSHDS,"[5.7770762, 7.997407]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",DS-SH-DS,DS-SH-DS SH-LH,0,0,DSSHDS SHSHLH,0,Other Calls
1,0.932017,1.367130,SHSHLH,MGGY,0,BWY MGGY Call Combo 1 290719 PM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,SHSHLH,"[1.8122675, 5.094298]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",SH-LH,DS-SH-DS SH-LH,1,1,DSSHDS SHSHLH,0,Contains LH Segment
2,1.218085,1.308841,DS,MGGY,0,BWY MGGY Call Combo 1 300719 AM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,DS,"[9.353501, 10.055656]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",DS,DS SH-LH,2,0,DS USC,1,Other Calls


In [8]:
### Create unique number identifiers for each class of calls
label_dict = {lab:i for i, lab in enumerate(np.unique(call_df['simp'].values))}
call_df['simp_num'] = [label_dict[i] for i in call_df.simp.values]
call_df[:3]

,start_time,end_time,labels,indv,indvi,filename,group,location,sex,wav_loc,...,umap,spectrogram,call_lab_simp,combi_lab_simp,call_unique_num,call_pos_combi,combi_label,combi_unique_num,simp,simp_num
ID,,,,,,,,,,,,,,,,,,,,,
0,0.753604,0.921160,DSSHDS,MGGY,0,BWY MGGY Call Combo 1 290719 PM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,"[5.7770762, 7.997407]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",DS-SH-DS,DS-SH-DS SH-LH,0,0,DSSHDS SHSHLH,0,Other Calls,2
1,0.932017,1.367130,SHSHLH,MGGY,0,BWY MGGY Call Combo 1 290719 PM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,"[1.8122675, 5.094298]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",SH-LH,DS-SH-DS SH-LH,1,1,DSSHDS SHSHLH,0,Contains LH Segment,0
2,1.218085,1.308841,DS,MGGY,0,BWY MGGY Call Combo 1 300719 AM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,"[9.353501, 10.055656]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",DS,DS SH-LH,2,0,DS USC,1,Other Calls,2


In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.autonotebook import tqdm
from joblib import Parallel, delayed
import umap
import pandas as pd

### Generate 10,000 simulations of transition probabilities
1. Randomly shuffle column containing call labels (in call_df) such that the new combinations are generated which contain a random combination of calls. Overall, the number of calls in each class/group remains the same.
2. Calculate transition probabilities for calls within these randomly generated combinations.
3. Add the calculated transition probabilities to a dataframe.
4. Repeat the above three steps 10,000 times.

In [12]:
dfs = []
for randi in range(0,10000):
    call_df[randi] = np.random.permutation(call_df["simp_num"].values) 
    unique_states = np.unique(call_df[randi])
    lab_dict = {lab: i for i, lab in enumerate(unique_states)}
    
    combis = [
        list(call_df[call_df.combi_unique_num == combiid][randi].values)
        for combiid in call_df.combi_unique_num.unique()
    ]
    # transition probs
    trans_mat = np.zeros((len(unique_states), len(unique_states)))
    for call in combis:
        for i, j in zip(call[:-1], call[1:]):
            trans_mat[i, j] += 1
    # smooth to nonzero probabilities
    trans_mat = (trans_mat.T / trans_mat.sum(axis=1)).T  # np.sum(trans_mat, axis=1)
    
    flat_mat = trans_mat.flatten()

    df = pd.DataFrame([flat_mat], columns = [
        "0-0", "0-1", "0-2", "1-0", 
        "1-1", "1-2", "2-0", "2-1",
        "2-2"
    ])
    df["rand_run"] = [randi]
    
    dfs.append(df)
    
randdfs = pd.concat(dfs)
len(randdfs)

10000

In [13]:
randdfs

,0-0,0-1,0-2,1-0,1-1,1-2,2-0,2-1,2-2,rand_run
0,0.353448,0.068966,0.577586,0.428571,0.000000,0.571429,0.392344,0.038278,0.569378,0
0,0.362069,0.060345,0.577586,0.470588,0.000000,0.529412,0.368932,0.043689,0.587379,1
0,0.293651,0.047619,0.658730,0.538462,0.000000,0.461538,0.360000,0.060000,0.580000,2
0,0.375000,0.075000,0.550000,0.470588,0.000000,0.529412,0.321782,0.029703,0.648515,3
0,0.462185,0.042017,0.495798,0.600000,0.066667,0.333333,0.341463,0.058537,0.600000,4
...,...,...,...,...,...,...,...,...,...,...
0,0.401575,0.078740,0.519685,0.461538,0.076923,0.461538,0.336683,0.040201,0.623116,9995
0,0.393162,0.034188,0.572650,0.176471,0.176471,0.647059,0.331707,0.053659,0.614634,9996
0,0.357724,0.073171,0.569106,0.235294,0.058824,0.705882,0.381910,0.040201,0.577889,9997
0,0.375000,0.044118,0.580882,0.333333,0.000000,0.666667,0.308901,0.057592,0.633508,9998


In [14]:
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir

In [15]:
#save df
save_loc = DATA_DIR / DATASET_ID / 'Monte_Carlo_Calls' /  'random_combi_simulations.pickle'
ensure_dir(save_loc.as_posix())
randdfs.to_pickle(save_loc)